## Cleaning Eurostat data


### Let's start with electricity import and export per country

In [1]:
import pandas as pd
import numpy as np

df= pd.read_csv('Electricity import export transformation per country.tsv', sep='\t')
df = df.set_index(df.columns[0])
df = df.transpose()
df.index.name = 'date'

#We will keep just the columns with import, export and electricity available to market
df.rename(columns=lambda col: col.replace('AIM,', 'Electricity available to market GWh'), inplace=True)
df.rename(columns=lambda col: col.replace('IMP,', 'Electricity imported GWh'), inplace=True)
df.rename(columns=lambda col: col.replace('EXP,', 'Electricity exported GWh'), inplace=True)
df = df.loc[:,~df.columns.str.endswith('20')]
df = df.loc[:,df.columns.str.contains('GWh')]

#Giving better names
df.rename(columns=lambda col: col.replace('M,', ''), inplace=True)
df.rename(columns=lambda col: col.replace('E7000,GWH,', ''), inplace=True)

# Multiindexing for date and country

## Mapping dictionary for European countries and adjacent countries
country_mapping = {
    'AL': 'Albania',
    'AT': 'Austria',
    'BA': 'Bosnia and Herzegovina',
    'BE': 'Belgium',
    'BG': 'Bulgaria',
    'CH': 'Switzerland',
    'CY': 'Cyprus',
    'CZ': 'Czech Republic',
    'DE': 'Germany',
    'DK': 'Denmark',
    'EE': 'Estonia',
    'EL': 'Greece',
    'ES': 'Spain',
    'FI': 'Finland',
    'FR': 'France',
    'HR': 'Croatia',
    'HU': 'Hungary',
    'IE': 'Ireland',
    'IS': 'Iceland',
    'IT': 'Italy',
    'LI': 'Liechtenstein',
    'LT': 'Lithuania',
    'LU': 'Luxembourg',
    'LV': 'Latvia',
    'ME': 'Montenegro',
    'MK': 'North Macedonia',
    'MT': 'Malta',
    'NL': 'Netherlands',
    'NO': 'Norway',
    'PL': 'Poland',
    'PT': 'Portugal',
    'RO': 'Romania',
    'RU': 'Russia',
    'RS': 'Serbia',
    'SE': 'Sweden',
    'SI': 'Slovenia',
    'SK': 'Slovakia',
    'TR': 'Turkey',
    'UK': 'United Kingdom',
    'XK': 'Kosovo',
    'UA': 'Ukraine',
    'GE': 'Georgia',
    'MD': 'Moldova'
}

#Creating a multiindex
tuples = []

for i in country_mapping:
    for j in df.index:
        tuples.append((i,j))
        
mi = pd.MultiIndex.from_tuples(tuples, names=('country', 'date'))

#New dataframe in which I add the data
s = pd.DataFrame(np.random.randn(len(mi),3), index=mi, columns=['Electricity import (GWh)','Electricity available to market (GWh)','Electricity export (GWh)'])

#Putting data in the new dataframe
kt = []
for k in df.columns:
    kt.append(k[:-2])
kt = list(set(kt))

ci = 0
for k in kt:
    c = s.columns[ci]
    for i in country_mapping:
        st = k + i
        for j in df.index:
            if st in df.columns:
                s.loc[(i,j),c] = df.loc[j,st]
            else:
                s.loc[(i,j),c] = ':'

    ci = ci + 1

display(s)


Electricity import (GWh)  \
country date                                
AL      2008-01                        :    
        2008-02                        :    
        2008-03                        :    
        2008-04                        :    
        2008-05                        :    
...                                   ...   
MD      2023-10                   56.098    
        2023-11                   34.553    
        2023-12                   31.656    
        2024-01                   32.177    
        2024-02                        :    

                 Electricity available to market (GWh)  \
country date                                             
AL      2008-01                                     :    
        2008-02                                     :    
        2008-03                                     :    
        2008-04                                     :    
        2008-05                                     :    
...                                                ...   
MD      2023-10                               341.081    
        2023-11                               311.614    
        2023-12                               316.417    
        2024-01                               303.787    
        2024-02                                     :    

                 Electricity export (GWh)  
country date                               
AL      2008-01                        :   
        2008-02                        :   
        2008-03                        :   
        2008-04                        :   
        2008-05                        :   
...                                   ...  
MD      2023-10                  303.988   
        2023-11                  304.493   
        2023-12                  347.600   
        2024-01                  392.548   
        2024-02                        :   

[8342 rows x 3 columns]

### Electricity mix per country

In [2]:
df= pd.read_csv('Electricity mix per country.tsv', sep='\t')
df = df.set_index(df.columns[0])
df = df.transpose()
df.index.name = 'date'

#Getting rid of predicted and estimated labels
df = df.replace('e', '')
df = df.replace('p', '')
df[:] = df[:].replace(' ', '')

#Giving better names
df.rename(columns=lambda col: col.replace('M,', ''), inplace=True)
df.rename(columns=lambda col: col.replace('GWH,', ''), inplace=True)
df.rename(columns=lambda col: col.replace('G3000', 'Electricity generated from natural gas GWh'), inplace=True)

#We generate the following information: Electricity generated in total, reliance on gas for production
summingtitles = ['CF','RA100','RA200','RA300','RA400','RA500_5160','N9000']

display(df)


"freq,siec,unit,geo\TIME_PERIOD","C0000,AL","C0000,AT","C0000,BA","C0000,BE","C0000,BG","C0000,CY","C0000,CZ","C0000,DE","C0000,DK","C0000,EE",...,"X9900,PT","X9900,RO","X9900,RS","X9900,SE","X9900,SI","X9900,SK","X9900,TR","X9900,UA","X9900,UK","X9900,XK"
date,,,,,,,,,,,,,,,,,,,,,
2016-01,:,:,:,:,:,:,:,:,:,:,...,:,:,:,:,:,:,:,:,:,:
2016-02,:,:,:,:,:,:,:,:,:,:,...,:,:,:,:,:,:,:,:,:,:
2016-03,:,:,:,:,:,:,:,:,:,:,...,:,:,:,:,:,:,:,:,:,:
2016-04,:,:,:,:,:,:,:,:,:,:,...,:,:,:,:,:,:,:,:,:,:
2016-05,:,:,:,:,:,:,:,:,:,:,...,:,:,:,:,:,:,:,:,:,:
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10,0.000,147.864,796.937,3.602,781.035,0.000,2610.974,9674.588 e,44.275,0.000,...,0.000,0.000,0.000,0.000,0.000,1.000,85.657,:,:,0.000
2023-11,0.000,146.045,692.416,1.176,871.225,0.000,2753.006,10947.152 e,226.176,0.000,...,0.000,0.000,0.000,0.000,0.000,1.000,79.285,:,:,:
2023-12,0.000,158.347,685.594,1.002,1043.759,0.000,2762.875,10597.592 e,257.861,0.000,...,0.000,0.000,0.000,0.000,0.000,1.000,76.049,:,:,:


In [3]:
totalgeneration = pd.DataFrame(0, index=mi, columns=['Total electricity generated GWh']) 
gasgenerationfraction = pd.DataFrame(np.nan, index=mi, columns=['Fraction of electricity generated by gas']) 

for i in country_mapping:
    for k in summingtitles:
        st = k + ',' + i
        if st in df.columns:
                for j in df.index:
                        #this cleans p, n e labels
                        df.loc[j,st] = df.loc[j,st].replace(' ', '')
                        df.loc[j,st] = df.loc[j,st].replace('e', '')
                        df.loc[j,st] = df.loc[j,st].replace('p', '')
                        df.loc[j,st] = df.loc[j,st].replace('n', '')
                        df.loc[j,st] = df.loc[j,st].replace('z', '')
                        df.loc[j,st] = df.loc[j,st].replace('u', '')
                        df.loc[j,st] = df.loc[j,st].replace('d', '')
                        #Making the new feature
                        if (df.loc[j,st] != ':') and (df.loc[j,st] != ': ') and (totalgeneration.loc[i,j] != [':']).all():    
                                totalgeneration.loc[(i,j),'Total electricity generated GWh'] = totalgeneration.loc[(i,j),'Total electricity generated GWh'] + float(df.loc[j,st])
                        else:
                                totalgeneration.loc[i,j] = ':'

        else:
                totalgeneration.loc[i] = [':']

totalgeneration.replace(0, np.nan, inplace=True)
                
for i in country_mapping:
        st1 = 'Electricity generated from natural gas GWh,' + i
        if st1 in df.columns:
                for j in df.index:
                        #this cleans p, n e labels
                        df.loc[j,st1] = df.loc[j,st1].replace(' ', '')
                        df.loc[j,st1] = df.loc[j,st1].replace('e', '')
                        df.loc[j,st1] = df.loc[j,st1].replace('p', '')
                        df.loc[j,st1] = df.loc[j,st1].replace('n', '')
                        df.loc[j,st1] = df.loc[j,st1].replace('z', '')
                        df.loc[j,st1] = df.loc[j,st1].replace('u', '')
                        df.loc[j,st1] = df.loc[j,st1].replace('d', '')
                        df.loc[j,st1] = df.loc[j,st1].replace('c', '')
                        #Putting ele-gas data in s
                        if st1 in df.columns:
                                s.loc[(i,j),'Electricity generated from natural gas GWh'] = df.loc[j,st1]
                        else:
                                s.loc[(i,j),'Electricity generated from natural gas GWh'] = ':'
                        #Making the new feature
                        if (df.loc[j,st1]!= ':') and (df.loc[j,st1]!= ': ') and (totalgeneration.loc[(i,j),'Total electricity generated GWh']!= ':') and (s.loc[(i,j),'Electricity generated from natural gas GWh']!= ':'):    
                                gasgenerationfraction.loc[(i,j),'Fraction of electricity generated by gas'] = float(df.loc[j,st1]) / totalgeneration.loc[(i,j),'Total electricity generated GWh']
                        else:
                                gasgenerationfraction.loc[(i,j),'Fraction of electricity generated by gas'] = ':'

#We only keep the information on gas
df = df.loc[:,~df.columns.str.endswith('20')]
df = df.loc[:,df.columns.str.contains('GWh')]

#Merging and cleaning
s = pd.merge(s,totalgeneration, left_index=True, right_index=True)
s = pd.merge(s,gasgenerationfraction, left_index=True, right_index=True)
s.replace(':', np.nan, inplace=True)
s.replace(': ', np.nan, inplace=True)

display(s)

Electricity import (GWh)  \
country date                                
AL      2008-01                       NaN   
        2008-02                       NaN   
        2008-03                       NaN   
        2008-04                       NaN   
        2008-05                       NaN   
...                                   ...   
MD      2023-10                   56.098    
        2023-11                   34.553    
        2023-12                   31.656    
        2024-01                   32.177    
        2024-02                       NaN   

                 Electricity available to market (GWh)  \
country date                                             
AL      2008-01                                    NaN   
        2008-02                                    NaN   
        2008-03                                    NaN   
        2008-04                                    NaN   
        2008-05                                    NaN   
...                                                ...   
MD      2023-10                               341.081    
        2023-11                               311.614    
        2023-12                               316.417    
        2024-01                               303.787    
        2024-02                                    NaN   

                 Electricity export (GWh)  \
country date                                
AL      2008-01                       NaN   
        2008-02                       NaN   
        2008-03                       NaN   
        2008-04                       NaN   
        2008-05                       NaN   
...                                   ...   
MD      2023-10                  303.988    
        2023-11                  304.493    
        2023-12                  347.600    
        2024-01                  392.548    
        2024-02                       NaN   

                 Electricity generated from natural gas GWh  \
country date                                                  
AL      2008-01                                         NaN   
        2008-02                                         NaN   
        2008-03                                         NaN   
        2008-04                                         NaN   
        2008-05                                         NaN   
...                                                     ...   
MD      2023-10                                      25.592   
        2023-11                                      73.918   
        2023-12                                     120.680   
        2024-01                                     126.431   
        2024-02                                         NaN   

                  Total electricity generated GWh  \
country date                                        
AL      2008-01                               NaN   
        2008-02                               NaN   
        2008-03                               NaN   
        2008-04                               NaN   
        2008-05                               NaN   
...                                           ...   
MD      2023-10                            37.600   
        2023-11                            80.085   
        2023-12                           126.124   
        2024-01                           142.357   
        2024-02                               NaN   

                  Fraction of electricity generated by gas  
country date                                                
AL      2008-01                                        NaN  
        2008-02                                        NaN  
        2008-03                                        NaN  
        2008-04                                        NaN  
        2008-05                                        NaN  
...                                                    ...  
MD      2023-10                                   0.680638  
        2023-11                

### Gas imports, exports and consumption

In [4]:
df= pd.read_csv('Gas import, export, consumption per country.tsv', sep='\t')
df = df.set_index(df.columns[0])
df = df.transpose()
df.index.name = 'date'

#We will keep just the columns with import, export and electricity available to market
df.rename(columns=lambda col: col.replace('IPRD,', 'Gas produced Mm3'), inplace=True)
df.rename(columns=lambda col: col.replace('TOS,', 'Gas from other sources Mm3'), inplace=True)
df.rename(columns=lambda col: col.replace('IMP,', 'Gas imported Mm3'), inplace=True)
df.rename(columns=lambda col: col.replace('EXP,', 'Gas exported Mm3'), inplace=True)
df.rename(columns=lambda col: col.replace('IC_OBS,', 'Gas consumed Mm3'), inplace=True)

df = df.loc[:,~df.columns.str.endswith('20')]
df = df.loc[:,df.columns.str.contains('Mm3')]
df = df.loc[:,~df.columns.str.contains('TJ_GCV,')]

#Giving better names
df.rename(columns=lambda col: col.replace('M,', ''), inplace=True)
df.rename(columns=lambda col: col.replace('G3000,', ''), inplace=True)
df.rename(columns=lambda col: col.replace('MIO_M3,', ''), inplace=True)

display(df)

"freq,nrg_bal,siec,unit,geo\TIME_PERIOD",Gas exported Mm3AL,Gas exported Mm3AT,Gas exported Mm3BE,Gas exported Mm3BG,Gas exported Mm3CY,Gas exported Mm3CZ,Gas exported Mm3DE,Gas exported Mm3DK,Gas exported Mm3EE,Gas exported Mm3EL,...,Gas from other sources Mm3PL,Gas from other sources Mm3PT,Gas from other sources Mm3RO,Gas from other sources Mm3RS,Gas from other sources Mm3SE,Gas from other sources Mm3SI,Gas from other sources Mm3SK,Gas from other sources Mm3TR,Gas from other sources Mm3UK,Gas from other sources Mm3XK
date,,,,,,,,,,,,,,,,,,,,,
2008-01,:,:,:,:,:,:,:,:,:,:,...,:,:,:,:,:,:,:,:,:,:
2008-02,:,:,:,:,:,:,:,:,:,:,...,:,:,:,:,:,:,:,:,:,:
2008-03,:,:,:,:,:,:,:,:,:,:,...,:,:,:,:,:,:,:,:,:,:
2008-04,:,:,:,:,:,:,:,:,:,:,...,:,:,:,:,:,:,:,:,:,:
2008-05,:,:,:,:,:,:,:,:,:,:,...,:,:,:,:,:,:,:,:,:,:
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10,0.000,229.524,2886.500,1478.627,0.000,18.004,1160.321,668.954,20.100,1002.828,...,:,0.000,0.000,0.000,0.000,:,0.000,0.000,:,:
2023-11,0.000,482.022,2872.100,1283.671,0.000,52.275,746.120,799.742,0.000,952.190,...,:,0.000,0.000,0.000,0.000,:,0.000,0.000,:,:
2023-12,0.000,608.401,3009.200,1230.721,0.000,31.821,811.852,719.219,0.000,875.503,...,:,0.000,0.000,0.000,0.000,:,0.000,0.000,:,:


In [5]:
#Putting data in the dataframe

kt = []
for k in df.columns:
    kt.append(k[:-2])
kt = list(set(kt))

for i in country_mapping:
        for k in kt:
                st1 = k + i
                if st1 in df.columns:
                        for j in df.index:
                                #this cleans p, n e labels
                                df.loc[j,st1] = df.loc[j,st1].replace(' ', '')
                                df.loc[j,st1] = df.loc[j,st1].replace('e', '')
                                df.loc[j,st1] = df.loc[j,st1].replace('p', '')
                                df.loc[j,st1] = df.loc[j,st1].replace('n', '')
                                df.loc[j,st1] = df.loc[j,st1].replace('z', '')
                                df.loc[j,st1] = df.loc[j,st1].replace('u', '')
                                df.loc[j,st1] = df.loc[j,st1].replace('d', '')
                                df.loc[j,st1] = df.loc[j,st1].replace('c', '')
                                #Putting gas data in s
                                if st1 in df.columns:
                                        s.loc[(i,j),k] = df.loc[j,st1]
                                else:
                                        s.loc[(i,j),k] = ':'

s.replace(':', np.nan, inplace=True)
s.replace(': ', np.nan, inplace=True)

display(s)

Electricity import (GWh)  \
country date                                
AL      2008-01                       NaN   
        2008-02                       NaN   
        2008-03                       NaN   
        2008-04                       NaN   
        2008-05                       NaN   
...                                   ...   
MD      2023-10                   56.098    
        2023-11                   34.553    
        2023-12                   31.656    
        2024-01                   32.177    
        2024-02                       NaN   

                 Electricity available to market (GWh)  \
country date                                             
AL      2008-01                                    NaN   
        2008-02                                    NaN   
        2008-03                                    NaN   
        2008-04                                    NaN   
        2008-05                                    NaN   
...                                                ...   
MD      2023-10                               341.081    
        2023-11                               311.614    
        2023-12                               316.417    
        2024-01                               303.787    
        2024-02                                    NaN   

                 Electricity export (GWh)  \
country date                                
AL      2008-01                       NaN   
        2008-02                       NaN   
        2008-03                       NaN   
        2008-04                       NaN   
        2008-05                       NaN   
...                                   ...   
MD      2023-10                  303.988    
        2023-11                  304.493    
        2023-12                  347.600    
        2024-01                  392.548    
        2024-02                       NaN   

                 Electricity generated from natural gas GWh  \
country date                                                  
AL      2008-01                                         NaN   
        2008-02                                         NaN   
        2008-03                                         NaN   
        2008-04                                         NaN   
        2008-05                                         NaN   
...                                                     ...   
MD      2023-10                                      25.592   
        2023-11                                      73.918   
        2023-12                                     120.680   
        2024-01                                     126.431   
        2024-02                                         NaN   

                  Total electricity generated GWh  \
country date                                        
AL      2008-01                               NaN   
        2008-02                               NaN   
        2008-03                               NaN   
        2008-04                               NaN   
        2008-05                               NaN   
...                                           ...   
MD      2023-10                            37.600   
        2023-11                            80.085   
        2023-12                           126.124   
        2024-01                           142.357   
        2024-02                               NaN   

                  Fraction of electricity generated by gas Gas produced Mm3  \
country date                                                                  
AL      2008-01                                        NaN              NaN   
        2008-02                                        NaN              NaN   
        2008-03                                        NaN              NaN   
        2008-04                                        NaN              NaN   
        2008-05                                        NaN              NaN   
...                        

### Gas prices per households

In [8]:
df= pd.read_csv('Gas prices for households.tsv', sep='\t')
df = df.set_index(df.columns[0])
df = df.transpose()
df.index.name = 'date'

#We will keep just the columns with price excluding tax and choose the metric for biggest consumers as it seems most representative and average price is available only for a short period of time.
df.rename(columns=lambda col: col.replace('KWH,X_TAX,EUR,', 'Gas price for households, no tax €'), inplace=True)
df = df.loc[:,df.columns.str.contains('€')]

df = df.loc[:,df.columns.str.contains('GJ_GE200,')]
df = df.loc[:,~df.columns.str.endswith('20')]

#Giving better names
df.rename(columns=lambda col: col.replace('S,4100,GJ_GE200,', ''), inplace=True)

display(df)

"freq,product,nrg_cons,unit,tax,currency,geo\TIME_PERIOD","Gas price for households, no tax €AL","Gas price for households, no tax €AT","Gas price for households, no tax €BA","Gas price for households, no tax €BE","Gas price for households, no tax €BG","Gas price for households, no tax €CZ","Gas price for households, no tax €DE","Gas price for households, no tax €DK","Gas price for households, no tax €EA","Gas price for households, no tax €EE",...,"Gas price for households, no tax €PL","Gas price for households, no tax €PT","Gas price for households, no tax €RO","Gas price for households, no tax €RS","Gas price for households, no tax €SE","Gas price for households, no tax €SI","Gas price for households, no tax €SK","Gas price for households, no tax €TR","Gas price for households, no tax €UA","Gas price for households, no tax €UK"
date,,,,,,,,,,,,,,,,,,,,,
2007-S1,:,:,:,:,:,:,:,0.0406,0.0385,:,...,:,:,:,:,0.0509,:,:,:,:,:
2007-S2,:,0.0384,:,0.0359,0.0274,0.0298,0.0428,0.0462,0.0394,0.0221,...,0.0302,0.0509,0.0227,:,0.051,0.0376,0.0344,0.0264,:,0.0294
2008-S1,:,0.0371,:,0.044,0.0299,0.0369,0.0435,0.0516,0.041,0.0267,...,0.0321,0.0535,0.0214,:,0.0506,0.0406,0.0336,0.0261,:,0.0371
2008-S2,:,0.0392,:,0.0553,0.0349,0.0446,0.0526,0.0532,0.047,0.0297,...,0.038,0.055,0.022,:,0.0577,0.0531,0.0365,0.0381,:,0.0402
2009-S1,:,0.0415,:,0.0452,0.0404,0.0408,0.0435,0.0388,0.0445,0.0323,...,0.0305,0.048,0.0173,:,0.0464,0.0513,0.0388,0.0317,:,0.0364
2009-S2,:,0.0393,:,0.0374,0.0298,0.0388,0.0381,0.0426,0.0379,0.0287,...,0.0335,0.0507,0.0144,:,0.0496,0.0397,0.0401,0.0249,:,0.0352
2010-S1,:,0.0393,:,0.0384,0.0311,0.0376,0.0376,0.0473,0.0371,0.0275,...,0.0323,0.0466,0.0149,:,0.0553,0.0441,0.0363,0.0265,:,0.035
2010-S2,:,0.0387,:,0.0405,0.0364,0.041,0.0412,0.0482,0.0402,0.0307,...,0.0367,0.0513,0.0143,:,0.0557,0.0483,0.0371,0.0272,:,0.0345
2011-S1,:,0.0455,0.0354,0.0458,0.0362,0.0428,0.0417,0.053,0.0414,0.0322,...,0.0348,0.0506,0.0147,:,0.0579,0.0442,0.0431,0.0237,:,0.0358


In [9]:
#Reindexing the dataframe
for i in range(len(df.index)):
       if df.index[i].endswith('S1 '):
              df.loc[df.index[i],'sdate'] = df.index[i][:-3] + '01-01'
       else:
              df.loc[df.index[i],'sdate'] = df.index[i][:-3] + '07-01'

df = df.set_index('sdate')
df.index = pd.to_datetime(df.index)

#Upsampling to get data for each month
df = df.resample('M').ffill()

df.index = df.index.strftime('%Y-%m') + ' '
df.index.name = 'date'

display(df)

#Putting data in the dataframe

for i in country_mapping:
                st = 'Gas price for households, no tax €' + i
                if st in df.columns:
                        for j in df.index:
                                #this cleans p, n e labels
                                df.loc[j,st] = df.loc[j,st].replace(' ', '')
                                df.loc[j,st] = df.loc[j,st].replace('e', '')
                                df.loc[j,st] = df.loc[j,st].replace('p', '')
                                df.loc[j,st] = df.loc[j,st].replace('n', '')
                                df.loc[j,st] = df.loc[j,st].replace('z', '')
                                df.loc[j,st] = df.loc[j,st].replace('u', '')
                                df.loc[j,st] = df.loc[j,st].replace('d', '')
                                df.loc[j,st] = df.loc[j,st].replace('c', '')
                                #Putting gas data in s
                                if st in df.columns:
                                        s.loc[(i,j),'Gas price for households, no tax €'] = df.loc[j,st]
                                else:
                                        s.loc[(i,j),'Gas price for households, no tax €'] = ':'

                #no need for an else statement, it puts NaN by default

s.replace(':', np.nan, inplace=True)
s.replace(': ', np.nan, inplace=True)

s = s.sort_index(ascending=True)
display(s)

"freq,product,nrg_cons,unit,tax,currency,geo\TIME_PERIOD","Gas price for households, no tax €AL","Gas price for households, no tax €AT","Gas price for households, no tax €BA","Gas price for households, no tax €BE","Gas price for households, no tax €BG","Gas price for households, no tax €CZ","Gas price for households, no tax €DE","Gas price for households, no tax €DK","Gas price for households, no tax €EA","Gas price for households, no tax €EE",...,"Gas price for households, no tax €PL","Gas price for households, no tax €PT","Gas price for households, no tax €RO","Gas price for households, no tax €RS","Gas price for households, no tax €SE","Gas price for households, no tax €SI","Gas price for households, no tax €SK","Gas price for households, no tax €TR","Gas price for households, no tax €UA","Gas price for households, no tax €UK"
date,,,,,,,,,,,,,,,,,,,,,
2007-01,:,:,:,:,:,:,:,0.0406,0.0385,:,...,:,:,:,:,0.0509,:,:,:,:,:
2007-02,:,:,:,:,:,:,:,0.0406,0.0385,:,...,:,:,:,:,0.0509,:,:,:,:,:
2007-03,:,:,:,:,:,:,:,0.0406,0.0385,:,...,:,:,:,:,0.0509,:,:,:,:,:
2007-04,:,:,:,:,:,:,:,0.0406,0.0385,:,...,:,:,:,:,0.0509,:,:,:,:,:
2007-05,:,:,:,:,:,:,:,0.0406,0.0385,:,...,:,:,:,:,0.0509,:,:,:,:,:
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03,:,0.1169,0.0503,0.0874,0.0780,0.0886,0.0906,0.1196,0.0934,0.0843,...,0.0643,0.0770,0.0486,0.0365,0.1349,0.0852,0.0458,0.0204,:,:
2023-04,:,0.1169,0.0503,0.0874,0.0780,0.0886,0.0906,0.1196,0.0934,0.0843,...,0.0643,0.0770,0.0486,0.0365,0.1349,0.0852,0.0458,0.0204,:,:
2023-05,:,0.1169,0.0503,0.0874,0.0780,0.0886,0.0906,0.1196,0.0934,0.0843,...,0.0643,0.0770,0.0486,0.0365,0.1349,0.0852,0.0458,0.0204,:,:


Electricity import (GWh)  \
country date                                
AL      2007-01                       NaN   
        2007-02                       NaN   
        2007-03                       NaN   
        2007-04                       NaN   
        2007-05                       NaN   
...                                   ...   
XK      2023-10                  198.320    
        2023-11                       NaN   
        2023-12                       NaN   
        2024-01                       NaN   
        2024-02                       NaN   

                 Electricity available to market (GWh)  \
country date                                             
AL      2007-01                                    NaN   
        2007-02                                    NaN   
        2007-03                                    NaN   
        2007-04                                    NaN   
        2007-05                                    NaN   
...                                                ...   
XK      2023-10                               243.327    
        2023-11                                    NaN   
        2023-12                                    NaN   
        2024-01                                    NaN   
        2024-02                                    NaN   

                 Electricity export (GWh)  \
country date                                
AL      2007-01                       NaN   
        2007-02                       NaN   
        2007-03                       NaN   
        2007-04                       NaN   
        2007-05                       NaN   
...                                   ...   
XK      2023-10                  513.611    
        2023-11                       NaN   
        2023-12                       NaN   
        2024-01                       NaN   
        2024-02                       NaN   

                 Electricity generated from natural gas GWh  \
country date                                                  
AL      2007-01                                         NaN   
        2007-02                                         NaN   
        2007-03                                         NaN   
        2007-04                                         NaN   
        2007-05                                         NaN   
...                                                     ...   
XK      2023-10                                       0.000   
        2023-11                                         NaN   
        2023-12                                         NaN   
        2024-01                                         NaN   
        2024-02                                         NaN   

                  Total electricity generated GWh  \
country date                                        
AL      2007-01                               NaN   
        2007-02                               NaN   
        2007-03                               NaN   
        2007-04                               NaN   
        2007-05                               NaN   
...                                           ...   
XK      2023-10                           468.603   
        2023-11                               NaN   
        2023-12                               NaN   
        2024-01                               NaN   
        2024-02                               NaN   

                  Fraction of electricity generated by gas Gas produced Mm3  \
country date                                                                  
AL      2007-01                                        NaN              NaN   
        2007-02                                        NaN              NaN   
        2007-03                                        NaN              NaN   
        2007-04                                        NaN              NaN   
        2007-05                                        NaN              NaN   
...                        

### Gas prices for industries

### Population